# 869 Project LightGBM - feature engirneering from lily's data Wendy, Jiamin, JO

In [ ]:
#pip install pandasql

In [1]:
#import sys
#!{sys.executable} -m pip install xgboost


import xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
import datetime 
import time

import pandasql as ps

In [2]:
#Import data

import pandas as pd
import numpy as np


train_value = pd.read_csv("C:/Users/jenny/OneDrive/Documents/MMA/MMA 869/Project/train_data_Lily.csv")
test = pd.read_csv("C:/Users/jenny/OneDrive/Documents/MMA/MMA 869/Project/test_lily.csv")

train_value.head(5)

#column_labels = list(train_label.columns.values)
#column_labels.remove("id")

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,0,0.0,2012-11-13,527.0,0,85.0,33.125828,-5.118154,27391,0,...,7,0,3,3,0,0,2,4,3,1
1,1,0.0,2011-03-05,1042.0,1978,2353.0,34.770717,-9.395642,32980,0,...,6,1,3,3,0,0,2,4,3,0
2,2,0.0,2011-03-27,959.0,0,2158.0,36.115056,-6.279268,826,0,...,6,1,0,0,3,3,2,2,1,0
3,3,10.0,2013-06-03,670.0,1639,68.0,37.147432,-3.187555,19173,0,...,6,1,3,3,7,2,2,1,1,0
4,4,0.0,2011-03-22,41.0,0,1782.0,36.164893,-6.099289,39926,0,...,6,1,2,2,0,0,2,4,3,1


In [3]:
#combine test value and label

#train = pd.merge(left=train_label, right=train_value, how='left', left_on='id', right_on='id')
#train
train = train_value.copy()
train

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,0,0.0,2012-11-13,527.0,0,85.0,33.125828,-5.118154,27391,0,...,7,0,3,3,0,0,2,4,3,1
1,1,0.0,2011-03-05,1042.0,1978,2353.0,34.770717,-9.395642,32980,0,...,6,1,3,3,0,0,2,4,3,0
2,2,0.0,2011-03-27,959.0,0,2158.0,36.115056,-6.279268,826,0,...,6,1,0,0,3,3,2,2,1,0
3,3,10.0,2013-06-03,670.0,1639,68.0,37.147432,-3.187555,19173,0,...,6,1,3,3,7,2,2,1,1,0
4,4,0.0,2011-03-22,41.0,0,1782.0,36.164893,-6.099289,39926,0,...,6,1,2,2,0,0,2,4,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,74240,0.0,2013-03-22,2037.0,1183,1378.0,37.007726,-3.280868,38695,0,...,6,1,0,0,9,6,1,1,1,0
59396,74242,0.0,2011-04-12,2125.0,0,2035.0,33.724987,-8.940758,22875,0,...,6,1,3,3,4,4,1,1,1,0
59397,74243,0.0,2012-11-13,755.0,1188,1440.0,33.963539,-1.429477,39845,0,...,6,1,3,3,6,4,1,2,1,1
59398,74246,50.0,2011-03-07,1990.0,1428,2238.0,35.630481,-7.710549,32980,0,...,6,1,2,2,7,2,2,1,1,1


In [4]:
#remove longitude 0 values, impute them as NaN
train.loc[train.longitude == 0, 'longitude'] = np.nan
train.longitude.isnull().sum()
#now has 1,812 missing data

1812

In [5]:
#impute missing data to median of the same location (subvillage, ward, lga)
train.longitude.fillna(train.groupby(['subvillage'])['longitude'].transform('median'), inplace = True)
train.longitude.fillna(train.groupby(['ward'])['longitude'].transform('median'), inplace = True)
train.longitude.fillna(train.groupby(['lga'])['longitude'].transform('median'), inplace = True)
train.longitude.isnull().sum()

0

In [6]:
#remove latitude 0 values, impute them as NaN
train.loc[train.latitude.round(0) == -0.0, 'latitude'] = np.nan
train.latitude.isnull().sum()
#now has 1,812 missing data

1812

In [7]:
#impute based on subvillage
train.latitude.fillna(train.groupby(['subvillage'])['latitude'].transform('median'), inplace = True)
train.latitude.fillna(train.groupby(['ward'])['latitude'].transform('median'), inplace = True)
#impute based on lga
train.latitude.fillna(train.groupby(['lga'])['latitude'].transform('median'), inplace = True)
train.latitude.isnull().sum()

0

In [ ]:
# Ord_enc_2 = OrdinalEncoder()
# Ord_enc_2 = OrdinalEncoder(categories=['per bucket','monthly','annually','on failure','unknown','never pay'])
# train.loc[train.payment_type == 'per bucket', 'payment_type_1'] = '0'
# train.loc[train.payment_type == 'monthly', 'payment_type_1'] = '1'
# train.loc[train.payment_type == 'annually', 'payment_type_1'] = '2'
# train.loc[train.payment_type == 'on failure', 'payment_type_1'] = '3'
# train.loc[train.payment_type == 'unknown', 'payment_type_1'] = '4'
# train.loc[train.payment_type == 'never pay', 'payment_type_1'] = '5'
# print(ps.sqldf("select payment_type_1,count(*) from train group by payment_type_1"))

In [ ]:
# ############# quality
# # 'good' has the highest number of all target -> create individual dummy

# train['quality_group'] = np.where(train['quality_group'] == 'good',1,0)

In [ ]:
############# quantity


# Ord_enc_2 = Ord_enc_2.fit(train[['quantity']])
# train['quantity'] = Ord_enc_2.transform(train[['quantity']])

In [8]:
############ construction year
# bin years into new and old and create unknown for all the year = 0

def construction(df):
    if df['construction_year'] >= 1986:
        val = 'New'
    elif df['construction_year'] == 0:
        val = 'Unknown'
    else:
        val = 'Old'
    return val

train['construction_year'] = train.apply(construction, axis=1)

# Ord_enc_3 = OrdinalEncoder()
# Ord_enc_3 = Ord_enc_3.fit(train[['construction_year']])
# train['construction_year'] = Ord_enc_3.transform(train[['construction_year']])

In [9]:
############ permit 
# missing imputation and create dummy to indicate whether the value was imputed or not

imp = SimpleImputer(missing_values = np.nan, strategy='median') 
train['permit'] = imp.fit_transform(train[['permit']]).ravel()

train['permit'] = np.where(train['permit'].isna(),1,0)

In [ ]:
# #impute missing values
# cateogry_columns=train.select_dtypes(include=['object']).columns.tolist()
# integer_columns=train.select_dtypes(include=['int64','float64']).columns.tolist()

# for column in train:
#     if train[column].isnull().any():
#         if(column in cateogry_columns):
#             train[column]=train[column].fillna(train[column].mode()[0])
#         else:
#             train[column]=train[column].fillna(train[column].mean)

# train

In [ ]:
# #encoding target and categorical variables
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# train_y = label_encoder.fit_transform(train['status_group'])

# train_str = train.select_dtypes(include = ['object'])
# train_str = train_str.drop(['status_group'], axis=1)
# train_num = train.select_dtypes(exclude = ['object'])

# for feat in train_str:
#     train_str[feat] = label_encoder.fit_transform(train_str[feat].astype(str))
# # #print(enc_train_str)

# enc_train = pd.concat([train_num,train_str], axis=1)

# enc_train.head(5)
# #train_str.info()

In [10]:
# Split the dataframe into X (features) and y (target, aka, label).
X = train.drop(['status_group','id','date_recorded','management_group','extraction_type_group','extraction_type','source_type','source_class','waterpoint_type_group','payment_type','scheme_management'], axis=1)
y = train['status_group']


In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   funder                 59400 non-null  float64
 2   gps_height             59400 non-null  int64  
 3   installer              59400 non-null  float64
 4   longitude              59400 non-null  float64
 5   latitude               59400 non-null  float64
 6   wpt_name               59400 non-null  int64  
 7   num_private            59400 non-null  int64  
 8   basin                  59400 non-null  int64  
 9   subvillage             59400 non-null  float64
 10  region                 59400 non-null  int64  
 11  region_code            59400 non-null  int64  
 12  district_code          59400 non-null  int64  
 13  lga                    59400 non-null  int64  
 14  ward                   59400 non-null  int64  
 15  po

In [12]:
#LightGBM

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import lightgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

In [13]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.1.


In [14]:
# LightGBM can handle categorical features by itself. We just need to make 
# sure the dataframe is using panda's built-in "category" data type.
cat_cols = ['waterpoint_type','source','quantity','water_quality','payment','management','extraction_type_class','ward','lga','region','basin','funder','region_code','district_code','quality_group','installer','wpt_name','subvillage','recorded_by','scheme_name','construction_year']
X[cat_cols] = X[cat_cols].astype('category')

max_depth: 6, num_leaves: 63, n_estimators: 200, learning_rate: 0.03, max_bin: 127-- CV Score mean: 0.6630 


In [15]:
params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.03,
      "max_depth": 10,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 200,
      "num_leaves": 63,
      "path_smooth": 0,
      "reg_alpha":  0.05,
      "reg_lambda": 0.05,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 127,
      "extra_trees": False,
      "is_unbalance": True,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
}
        
fit_params= {
    'feature_name': "auto",
    'categorical_feature': cat_cols,
}

start = time.time()
estimator = LGBMClassifier(**params)
estimator = estimator.fit(X, y, **fit_params)

C:\Users\jenny\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


In [16]:
# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(estimator, X, y, 
                                 fit_params=fit_params, 
                                 cv=15, 
                                 scoring="f1_macro", 
                                 n_jobs=5, 
                                 verbose=0, 
                                 return_train_score=True,
                                error_score = "raise")
cv_scores=inner_cv_scores['test_score'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Fit times: {}".format(duration))
print("Total duration: {}".format(duration))

CV Scores:
['0.6710', '0.6526', '0.6753', '0.6639', '0.6698', '0.6797', '0.6578', '0.6713', '0.6506', '0.6684', '0.6608', '0.6573', '0.6443', '0.6548', '0.6682']
CV Score mean: 0.6630 
CV Score range: 0.6533 -- 0.6727
Fit times: 85.36900091171265
Total duration: 85.36900091171265


Now using the Test set
submission score Nov 20: 0.7943

In [17]:
#remove longitude 0 values, impute them as NaN
test.loc[test.longitude == 0, 'longitude'] = np.nan
test.longitude.isnull().sum()
#now has 1,812 missing data

457

In [18]:
#impute missing data to median of the same location (subvillage, ward, lga)
test.longitude.fillna(test.groupby(['subvillage'])['longitude'].transform('median'), inplace = True)
test.longitude.fillna(test.groupby(['ward'])['longitude'].transform('median'), inplace = True)
test.longitude.fillna(test.groupby(['lga'])['longitude'].transform('median'), inplace = True)
test.longitude.isnull().sum()

0

In [19]:
#remove latitude 0 values, impute them as NaN
test.loc[test.latitude.round(0) == -0.0, 'latitude'] = np.nan
test.latitude.isnull().sum()
#now has 1,812 missing data

457

In [20]:
#impute based on subvillage
test.latitude.fillna(test.groupby(['subvillage'])['latitude'].transform('median'), inplace = True)
test.latitude.fillna(test.groupby(['ward'])['latitude'].transform('median'), inplace = True)
#impute based on lga
test.latitude.fillna(test.groupby(['lga'])['latitude'].transform('median'), inplace = True)
test.latitude.isnull().sum()

0

In [ ]:
# Ord_enc_2 = OrdinalEncoder()
# Ord_enc_2 = OrdinalEncoder(categories=['per bucket','monthly','annually','on failure','unknown','never pay'])
# test.loc[test.payment_type == 'per bucket', 'payment_type_1'] = '0'
# test.loc[test.payment_type == 'monthly', 'payment_type_1'] = '1'
# test.loc[test.payment_type == 'annually', 'payment_type_1'] = '2'
# test.loc[test.payment_type == 'on failure', 'payment_type_1'] = '3'
# test.loc[test.payment_type == 'unknown', 'payment_type_1'] = '4'
# test.loc[test.payment_type == 'never pay', 'payment_type_1'] = '5'
# print(ps.sqldf("select payment_type_1,count(*) from train group by payment_type_1"))


In [21]:
############# quality
# 'good' has the highest number of all target -> create individual dummy

test['quality_group'] = np.where(test['quality_group'] == 'good',1,0)

In [ ]:
test['construction_year'].head(5)

In [22]:
############ construction year
# bin years into new and old and create unknown for all the year = 0

def construction(df):
    if df['construction_year'] >= 1986:
        val = 'New'
    elif df['construction_year'] == 0:
        val = 'Unknown'
    else:
        val = 'Old'
    return val

test['construction_year'] = test.apply(construction, axis=1)

# Ord_enc_3 = OrdinalEncoder()
# Ord_enc_3 = Ord_enc_3.fit(train[['construction_year']])
# train['construction_year'] = Ord_enc_3.transform(train[['construction_year']])

In [23]:
############ permit 
# missing imputation and create dummy to indicate whether the value was imputed or not

imp = SimpleImputer(missing_values = np.nan, strategy='median') 
test['permit'] = imp.fit_transform(test[['permit']]).ravel()

test['permit'] = np.where(test['permit'].isna(),1,0)

In [ ]:
# #impute missing values
# cateogry_columns=test.select_dtypes(include=['object']).columns.tolist()
# integer_columns=test.select_dtypes(include=['int64','float64']).columns.tolist()

# for column in test:
#     if test[column].isnull().any():
#         if(column in cateogry_columns):
#             test[column]=test[column].fillna(test[column].mode()[0])
#         else:
#             test[column]=test[column].fillna(test[column].mean)

In [ ]:
# #encoding target and categorical variables
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()

# test_str = test.select_dtypes(include = ['object'])
# test_num = test.select_dtypes(exclude = ['object'])

# for feat in test_str:
#     test_str[feat] = label_encoder.fit_transform(test_str[feat].astype(str))
# # #print(enc_train_str)

# enc_test = pd.concat([test_num,test_str], axis=1)

# enc_test.head(5)

test.info()

In [24]:
X_test = test.drop(['id','date_recorded','management_group','extraction_type_group','extraction_type','source_type','source_class','waterpoint_type_group','payment_type','scheme_management'], axis=1)
#cat_cols = ['waterpoint_type','source','quantity','water_quality','payment','management','extraction_type','ward','lga','region','basin','funder']
X_test[cat_cols] = X_test[cat_cols].astype('category')

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   amount_tsh             14850 non-null  float64 
 1   funder                 14850 non-null  category
 2   gps_height             14850 non-null  int64   
 3   installer              14850 non-null  category
 4   longitude              14850 non-null  float64 
 5   latitude               14850 non-null  float64 
 6   wpt_name               14850 non-null  category
 7   num_private            14850 non-null  int64   
 8   basin                  14850 non-null  category
 9   subvillage             14850 non-null  category
 10  region                 14850 non-null  category
 11  region_code            14850 non-null  category
 12  district_code          14850 non-null  category
 13  lga                    14850 non-null  category
 14  ward                   14850 non-null 

In [25]:

start = time.time()
test_y = estimator.predict(X_test)

In [26]:
test_y

predicted_y = pd.DataFrame(test_y, columns=['status_group'])

predicted_y.groupby('status_group').size()

array([1, 0, 1, ..., 0, 0, 1], dtype=int64)

status_group
0    9444
1    4959
2     447
dtype: int64

In [27]:
predicted_test = pd.concat([test['id'],predicted_y], axis=1)
predicted_test['status_group'] = predicted_test['status_group'].replace({0:'functional',2:'functional needs repair',1:'non functional'})
predicted_test.head(5)

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [28]:
#export test Y variables
submission = pd.read_csv("C:/Users/jenny/OneDrive/Documents/MMA/MMA 869/Project/Submission_format.csv")

submission.head(5)

,id,status_group
0,50785,predicted label
1,51630,predicted label
2,17168,predicted label
3,45559,predicted label
4,49871,predicted label


In [29]:
submission = pd.merge(left=submission['id'], right=predicted_test, how='left', left_on='id', right_on='id')

submission.head(5)

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [30]:
submission.to_csv('C:/Users/jenny/OneDrive/Documents/MMA/MMA 869/Project/Submission_LightGBM-lilydataJOWFJMfeatures.csv', index = False)